# Chemical Equation Balancer

The following code will compute the balanced form of the given chemical equation. The main purpose of this project is to get use to _numpy_ library and understanding the first use case of solving linear equations.

## Import required libraries

In [14]:
import numpy as np

## Get and Parse Inputs
Get and parse inputs to a useable format. 

(In the following documentation, we will be describing the code using the following example:

Input:
```txt
C H O
C2 H6 + O2 -> C O2 + H2 O
```
)

The following piece of code will parse the given inputs as follows:  
   + _equation_elements_ = `['C', 'H', 'O']`
   + _chemical_equation_ = `C2 H6 + O2 -> C O2 + H2 O`

In [121]:
def get_and_parse_inputs():
    equation_elements = np.array(input().split())
    chemical_equation = input()

## Methods for extracting coefficients
The following methods, will extract coefficients from equation and construct the corresponding augmented matrix. The description of each method is as follows:

+ __extract_elements_coefficients_from_mixture__:
    + given available_elements (`['C', 'H', 'O']` in our example) and a mixture (like `C2 H6` for example), this method will compute and return the coeffient for each element in the given mixture. (will return `[2, 6, 0]` in above example)
       
       
+ __extract_elements_coefficients_from_summation__:
    + given available_elements (`['C', 'H', 'O']` in our example) and a summation (like `C2 H6 + O2` for example), this method will compute the coeffient for each mixture and append it as a new column to the result matrix. (will return `[[2, 0], [6, 0], [0, 2]]` in above example)


In [168]:
def extract_elements_coefficients_from_mixture(available_elements, mixture):
    mixture_elements = np.array(mixture.split())
    element_count = dict.fromkeys(available_elements, 0)
    for element in mixture_elements:
        # given "C5", this will increase "C" element's count by 5
        if len(element) > 1:
            element_count[element[0]] += int(element[1])
        else:
            element_count[element[0]] += 1
    
    
    result_coefficients = np.zeros(available_elements.size)
    for index, element in enumerate(available_elements):
        result_coefficients[index] = element_count[element]
    return result_coefficients

In [118]:
def extract_elements_coefficients_from_summation(available_elements, summation, is_negative=False):
    mixtures = np.array(summation.split('+'))
    
    for index, mixture in enumerate(mixtures):
        mixture_coefficients = extract_elements_coefficients_from_mixture(available_elements, mixture)
        if is_negative:
            mixture_coefficients *= -1
            
        if index == 0:
            result_coefficients = mixture_coefficients
        else:
            # this will append the computed coefficients (mixture_coefficients) 
            # as a new column to the current extracted matrix
            result_coefficients = np.c_[result_coefficients, mixture_coefficients]
            
    return result_coefficients

## Constucting the augmented matrix

+ __construct_augmented_matrix__:
    + Construct the augmented matrix by calling _extract_elements_coefficients_from_summation_ method for reactants and products and appending another column of zeros.
    + In our example:
        + _reactants_ = `C2 H6 + O2 `
        + _products_ = ` C O2 + H2 O`
        + will return `[[2, 0, -1, 0, 0], [6, 0, 0, -2, 0], [0, 2, -2, -1, 0]]`

In [120]:
def construct_augmented_matrix(available_elements, equation):
    reactants, products = equation.split('->')
    reactants_matrix = extract_elements_coefficients_from_summation(available_elements, reactants)
    products_matrix = extract_elements_coefficients_from_summation(available_elements, products, True)
    return np.c_[reactants_matrix, products_matrix, np.zeros(available_elements.size)]

## Compute echelon form

Compute a echelon form of the augmented matrix.

In [160]:
def sort_matrix_rows_by_leading_entries(A):
    assert len(A.shape) == 2
    result = A.astype(np.float64)
    
    topmost_row = 0
    for j in range(result.shape[1]):
        for i in range(topmost_row, result.shape[0]):
            if float(A[i, j]) != 0:
                # swap current row with the first untouched row
                result[[topmost_row, i]] = result[[i, topmost_row]]
                topmost_row += 1
                i -= 1
    return result

In [165]:
def compute_echelon_matrix(A):
    assert len(A.shape) == 2
    result = A.astype(np.float64)
    
    topmost_row = 0
    for j in range(result.shape[1]):
        result = sort_matrix_rows_by_leading_entries(result)
        if float(result[topmost_row][j]) == 0:
            # this row and all rows at below are 0 at this column
            continue
            
        # leading entry at this column is set to be at topmost_row-th,
        # so the rows at below must be scaled and then subtracted by current row
        for i in range(topmost_row + 1, result.shape[0]):
            if float(result[i][j]) == 0:
                # this row and all rows at below are 0 at this column
                break
            
            scaling_coefficient = result[topmost_row][j] / result[i][j]
            for col in range(j, result.shape[1]):
                # first scale the element, the subtract it by the candidate entry
                result[i][col] *= scaling_coefficient
                result[i][col] -= result[topmost_row][col]
                
        topmost_row += 1
        if topmost_row == result.shape[0]:
            break
    return result

## Compute reduced echelon form
Transform the computed echelon matrix to the reduced echelon form

In [170]:
def compute_reduced_echelon_form(A):
    pass